In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import os as os
import pandas as pd
from itertools import product
import numpy as np
import scipy.stats as ss

In [ ]:
filename = "../hfactory_magic_folders/bond_recommender/RFQ_Data_Challenge_HEC.csv" 
df = pd.read_csv(filename, low_memory = False)

# some naive exploration ()

In [ ]:
df.info()

In [ ]:
df[['maturity','Maturity']]

In [ ]:
df.head()

In [ ]:
df[df.ISIN == 'DE000A14J587'][['Instrument', 'Coupon', 'Maturity', 'AssumedMaturity']]

In [ ]:
df.nunique()

In [ ]:
df.Rating_Moodys.unique()

In [ ]:
df.Rating_SP.unique()

In [ ]:
df[['Maturity', 'AssumedMaturity']].dropna()

In [ ]:
df.B_Price.describe()

In [ ]:
df[df.B_Price == 0].Total_Traded_Volume_Natixis

In [ ]:
df.Total_Requested_Volume.describe()

In [ ]:
df.BloomIndustrySector.unique()

In [ ]:
df.B_Side.unique()

In [ ]:
df[df.B_Side == '0.000000']

In [ ]:
df[df.B_Side == '0.000000'].Total_Requested_Volume

In [ ]:
df[['MidModifiedDuration', 'AssumedMaturity']]

# clean the data

In [ ]:
# change data type to datetime 
from datetime import datetime
df_clean = df
df_clean[['Maturity', 'AssumedMaturity', 'Deal_Date', 'YTWDate']] = df_clean[['Maturity', 'AssumedMaturity', 'Deal_Date', 'YTWDate']].apply(pd.to_datetime)

In [ ]:
df_clean[['Maturity', 'AssumedMaturity', 'Deal_Date', 'YTWDate']]

In [ ]:
# add a column indicating the time to maturity (DAYS)
df_clean['TTM'] = df_clean['Maturity'] - df_clean['Deal_Date']
df_clean.TTM = df_clean.TTM.dt.days.astype('int16')

In [ ]:
df_clean.TTM

In [ ]:
# add a time of yeild to call YTCDays
df_clean['YTCDays'] = df_clean['YTWDate'] - df_clean['Deal_Date']
df['YTCDays'] = df['YTCDays'].fillna(pd.Timedelta(seconds=0))
df_clean.YTCDays = df_clean.YTCDays.dt.days.astype('int16')

In [ ]:
df_clean.YTCDays

In [ ]:
# drop rows with number of na >8
df_clean.dropna(thresh=27, inplace = True)

In [ ]:
# turn datetime into year, month, day
df_clean['Deal_Date_year'] = df_clean['Deal_Date'].dt.year
df_clean['Deal_Date_month'] = df_clean['Deal_Date'].dt.month
df_clean['Deal_Date_day'] = df_clean['Deal_Date'].dt.day
df_clean['Deal_Date_dayofweek'] = df_clean['Deal_Date'].dt.dayofweek

df_clean['Maturity_year'] = df_clean['Maturity'].dt.year
df_clean['Maturity_month'] = df_clean['Maturity'].dt.month
df_clean['Maturity_day'] = df_clean['Maturity'].dt.day
df_clean['Maturity_dayofweek'] = df_clean['Maturity'].dt.dayofweek

df_clean['YTWDate_year'] = df_clean['YTWDate'].dt.year
df_clean['YTWDate_month'] = df_clean['YTWDate'].dt.month
df_clean['YTWDate_day'] = df_clean['YTWDate'].dt.day
df_clean['YTWDate_dayofweek'] = df_clean['YTWDate'].dt.dayofweek

df_clean.drop(columns = ['Deal_Date', 'Maturity', 'YTWDate'], inplace = True)

In [ ]:
# drop columns that are not useful or have too many nas

df_clean = df_clean.drop(columns=['cusip','maturity', 'Cusip', 'Instrument', 'Sales_Name', 'Sales_Initial', 'Rating_Fitch', 'Rating_Moodys', 'Tier', 'AssumedMaturity'])

In [ ]:
df_clean.drop(df_clean[df_clean.B_Side == '0.000000'].index, inplace = True)

In [ ]:
df_clean.reset_index(drop=True)

In [ ]:
df_clean[['Total_Requested_Volume', 'B_Price']] = df_clean[['Total_Requested_Volume', 'B_Price']].astype(float)

In [ ]:
df_clean.info()

In [ ]:
cat_features = list(df_clean.select_dtypes(include=['object']).columns)
cat_features.remove('ISIN')
num_features = list(df_clean._get_numeric_data().columns)
num_features

# correlation heatmaps

## numerical

In [ ]:
df_num = df_clean[num_features]
# Increase the size of the heatmap.
plt.figure(figsize=(28, 12))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(df_num.corr(), vmin=-1, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

In [ ]:
# drop corr>0.7
df_num_new = df_num.drop(columns = ['MidModifiedDuration', 'Total_Traded_Volume', 'MidEffectiveDuration', 'YTWDate_month', 'YTWDate_day', 'YTWDate_dayofweek'
                               , 'MidYTM', 'SpreadvsBenchmarkMid', 'Maturity_year', 'YTWDate_year', 'MidASWSpread'])
# Increase the size of the heatmap
plt.figure(figsize=(28,12))
# Store heatmap object in a variable to easily access it when you want to include more features (such as title).
# Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
heatmap = sns.heatmap(df_num_new.corr(), vmin=-1, vmax=1, annot=True)
# Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
heatmap.set_title('New Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

## categorical 

In [ ]:
df_cat = df_clean[cat_features]
df_cat.head()

In [ ]:
## Removing records with at least one null value in a row
df_cat_v1 = df_cat.dropna()
df_cat_v1.shape
## Let us split this list into two parts
cat_var1 = (cat_features)
cat_var2 = (cat_features)
## Let us jump to Chi-Square test
## Creating all possible combinations between the above two variables list
cat_var_prod = list(product(cat_var1,cat_var2, repeat = 1))

In [ ]:
## Creating an empty variable and picking only the p value from the output of Chi-Square test
result = []
for i in cat_var_prod:
    if i[0] != i[1]:
        result.append((i[0],i[1],list(ss.chi2_contingency(pd.crosstab(df_cat_v1[i[0]], df_cat_v1[i[1]])))[1]))

In [ ]:
chi_test_output = pd.DataFrame(result, columns = ['var1', 'var2', 'coeff'])
## Using pivot function to convert the above DataFrame into a crosstab
chi_test_output.pivot(index='var1', columns='var2', values='coeff')

### cramers V

In [ ]:
from scipy.stats import chi2_contingency
import seaborn as sns

def cramerV(label,x):
    confusion_matrix = pd.crosstab(label, x)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r,k = confusion_matrix.shape
    phi2 = chi2/n
    phi2corr = max(0,phi2-((k-1)*(r-1))/(n-1))
    rcorr = r - ((r - 1) ** 2) / ( n - 1 )
    kcorr = k - ((k - 1) ** 2) / ( n - 1 )
    try:
        if min((kcorr - 1),(rcorr - 1)) == 0:
            warnings.warn(
            "Unable to calculate Cramer's V using bias correction. Consider not using bias correction",RuntimeWarning)
            v = 0
            print("If condition Met: ",v)
        else:
            v = np.sqrt(phi2corr / min((kcorr - 1), (rcorr - 1)))
            #print("Else condition Met: ",v)
    except:
        print("inside error")
        v = 0
    return v
def plot_cramer(df):
    cramer = pd.DataFrame(index=df.columns,columns=df.columns)
    for column_of_interest in df.columns:
        try:
            temp = {}

            columns = df.columns
            for j in range(0,len(columns)):
                v = cramerV(df[column_of_interest],df[columns[j]])
                cramer.loc[column_of_interest,columns[j]] = v
                if (column_of_interest==columns[j]):
                    pass
                else:
                    temp[columns[j]] = v
            cramer.fillna(value=np.nan,inplace=True)
        except:
            print('Dropping row:',column_of_interest)
            pass
    plt.figure(figsize=(16,6))
    sns.heatmap(cramer,annot=True,fmt='.2f')

    plt.title("Cross Correlation plot on Dataframe with Cramer's Correlation Values")
    plt.show()
       
plot_cramer(df_cat)    

In [ ]:
# drop columns that are highly correlated with others
df_clean.drop(columns = ['MidModifiedDuration', 'Total_Traded_Volume', 'MidEffectiveDuration', 'YTWDate_month', 'YTWDate_day', 'YTWDate_dayofweek'
                               , 'MidYTM', 'SpreadvsBenchmarkMid', 'Maturity_year', 'YTWDate_year', 'MidASWSpread', 'cdcissuer', 'cdcissuerShortName', 'lb_Platform_2', 'BloomIndustrySector', 'BloomIndustrySubGroup'], inplace = True)

In [ ]:
!pip install xgboost

In [ ]:
import json

with open('categorical-model.json') as user_file:
    file_contents = user_file.read()

print(file_contents)

parsed_json = json.loads(file_contents)

# Task1: define a similarity measurement for bonds

In [ ]:
# create a dataframe with bond features without operational informations
df_bonds = df_clean.drop(columns = ['Total_Traded_Volume_Natixis', 'Total_Traded_Volume_Away', 'Total_Requested_Volume', 'B_Side', 'B_Price', 'company_short_name'])

In [ ]:
# drop duplicates
df_bonds.drop_duplicates(inplace = True)
df_bonds.reset_index().drop=True

In [ ]:
df_bonds.info()

In [ ]:
df_bonds[df_bonds.select_dtypes(['object']).columns] = df_bonds.select_dtypes(['object']).apply(lambda x: x.astype('category'))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
# Identify numerical and categorical features
numeric_features = X.select_dtypes(include=['int', 'float']).columns
categorical_features = X.select_dtypes(include=['object']).columns

# Create transformers for numerical and categorical features
numeric_transformer = Pipeline(steps=[
    ('num', 'passthrough')  # For numerical features, we just pass them through
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Create the Random Forest Classifier
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = df_bonds.dropna().ISIN
X = df_bonds.dropna().drop(columns = ['ISIN'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=42)
y_train = le.fit_transform(y_train)

In [ ]:
# Supported tree methods are `approx` and `hist`.
clf = xgb.XGBClassifier(enable_categorical=True)
# X is the dataframe we created in previous snippet
clf.fit(X_train, y_train)
# Must use JSON/UBJSON for serialization, otherwise the information is lost.
clf.save_model("categorical-model.json")

In [ ]:
importance = clf.feature_importances_

# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
plt.bar([x for x in range(len(importance))], importance)
plt.show()

In [ ]:
importance.shape

In [ ]:
int_columns = df_bonds.select_dtypes(include='int').columns
df_bonds[int_columns] = df_bonds[int_columns].astype(float)

In [ ]:
# encode the categorical columns to 0, 1
df_encoded = pd.get_dummies(df_bonds.drop(columns = ['ISIN']), columns=df_bonds.select_dtypes(include='category').columns, dtype=float)

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

# Step 1: the last row as the new input
new_row = df_encoded.iloc[-1,:]

# Step 2: Specify the weights for each feature
weights = importance

# Step 3: Calculate the cosine similarity with weighted features
features = df_encoded.values

# Apply weights to the features
weighted_features = np.dot(features, weights)

# Normalize the weighted feature vectors to ensure accurate cosine similarity calculation
#normalized_weighted_features = normalize(weighted_features, axis=0)

# Calculate the cosine similarity matrix
cosine_sim_matrix = cosine_similarity(weighted_features, weighted_features)

# Step 4: Extract the similarity values for the new row
new_row_index = df_encoded.shape[0] - 1  # Index of the new row
similarities_with_existing_rows = cosine_sim_matrix[new_row_index, :-1]

# Step 5: Add the similarities to the DataFrame or use them as needed
df_encoded['CosineSimilarity'] = similarities_with_existing_rows

# Now df contains the cosine similarity values with weighted features for the new row with existing rows


# Task2: recommendation system for clients

Idea 1: Using collaborative filtering between bonds and clients
Each bond and each client are associated with a score (for example: frequency) in the pivot table
When a new bond came in, it can suggests clients who may have interests in.





In [ ]:
# associate clients and bonds with the number of requests
pivot_table_count = df_clean.pivot_table(index='company_short_name', columns='ISIN', values=['Total_Requested_Volume'], aggfunc = ['count'],  fill_value=0)

# Display the pivot table
print(pivot_table_count)

In [ ]:
pivot_table_count.columns = pivot_table_count.columns.droplevel(0) #remove amount
df1 = pivot_table_count.reset_index().rename_axis(None, axis=1)
df1

In [ ]:
def interested_customers(pivot_table_df, bond_name, top_n = 5):
    if (df1[bond_name] != 0).sum() < top_n:
        return list(df1.iloc[(df1[bond_name]!=0).index].company_short_name)
    else:
        return list(df1.iloc[df1[bond_name].nlargest(top_n).index].company_short_name)